In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [11]:
class Graph:
    def __init__(self, adj_matrix = None, edge_list = None):
        if adj_matrix:
            self.adj_matrix = pd.DataFrame(adj_matrix)
        else:
            self.adj_matrix = None
                
        self.edge_list = edge_list
        self.adj_list = defaultdict(list)
        
        if not self.adj_list and self.edge_list:
            self.prepare_adj_list()
        
        
    def addEdge(self, u, v, w):
        if not self.edge_list:
            self.edge_list = []
        self.edge_list.append([u, v, w])
        self.adj_list[u].append((v,w))
        self.adj_list[v].append((u,w))
    
    def prepare_adj_list(self):
        for u,v,w in self.edge_list:
            self.adj_list[u].append((v,w))
            self.adj_list[v].append((u,w))
            
    def prepare_adj_matrix(self):
        self.vertices = sorted(self.adj_list.keys())
        self.adj_matrix = pd.DataFrame(index=self.vertices, columns=self.vertices)
        #print(self.adj_matrix)
        
        for k,v in self.adj_list.items():
            for x in v:
                self.adj_matrix.loc[k][x[0]] = x[1]
        self.adj_matrix.fillna(0, inplace=True)
        #print(self.adj_matrix)
    
    def kruskal_MST(self):
        self.edge_list = sorted(self.edge_list,
                            key=lambda item: item[2])
        
        print('Sorted Input Edge List:\n')
        edge_df = pd.DataFrame(self.edge_list, columns=['Edge', 'Edge2', 'Cost'])
        edge_df['Edge'] = edge_df['Edge'].astype(str) + '-' + edge_df['Edge2'].astype(str)
        edge_df.drop(columns=['Edge2'], inplace=True)
        display(edge_df)
        print('\n\nApplying Kruskal Algorithm\n')
        q = []
        w_dict = defaultdict(list)
        for edge in self.edge_list:
            heapq.heappush(q, edge[2])
            w_dict[edge[2]].append((edge[0], edge[1]))
#         print(q)
#         print(w_dict)
        cluster_dict = [[key] for key in self.adj_list.keys()]
#         for key in self.adj_list.keys():
#             cluster_dict[key].append(key)
        print(f'\nClusters: {cluster_dict}')
        
        num_vertices = len(self.adj_list)
        mst_edges = []
        mst_cost = []
        while len(mst_edges) < num_vertices -1:
            w = heapq.heappop(q)
            u,v = w_dict[w][0]
            w_dict[w].remove((u,v))
            print(f'\nEdge: {u}-{v}')
            for i,x in enumerate(cluster_dict):
                if u in x:
                    c_u = x
                    c_u_i = i
                if v in x:
                    c_v = x
                    c_v_i = i
            if c_u !=c_v:
                print(f'\n{u} and {v} are in different clusters')
                print(f'\nAdd edge {u}-{v} to MST')
                print(f'\nMerge clusters of {u} and {v}')
                mst_edges.append(f'{u}-{v}')
                mst_cost.append(w)
                c_u.extend(c_v)
                del cluster_dict[c_v_i]
                print(f'\nClusters: {cluster_dict}')
            else:
                print(f'\n{u} and {v} are in same cluster, so reject edge {u}-{v}')
            print('----------------------------------------------------------')
        
        mst_df = pd.DataFrame(np.array([mst_edges, mst_cost]).T, columns=['Edge', 'Cost'])
        print('\n\nMST Edge List after applying Kruskal Algorithm:\n')
        display(mst_df)
        
        print(f'\nMST Cost: {mst_df.Cost.astype(int).sum()}')
        
    def prim_MST(self):
        if  self.adj_matrix is None:
            self.prepare_adj_matrix()
        
        
        print('Adjacency Matrix:\n')
        display(self.adj_matrix)
        
        N = self.adj_matrix.shape[0]
        selected_node = [0]*N

        no_edge = 0

        selected_node[0] = True

        # printing for edge and weight
        #df_mst = pd.
        
        G = self.adj_matrix.values
        mst_edges = []
        mst_cost = []
        while (no_edge < N - 1):

            minimum = np.inf
            a = 0
            b = 0
            for m in range(N):
                if selected_node[m]:
                    for n in range(N):
                        if ((not selected_node[n]) and G[m][n]):  
                            # not in selected and there is an edge
                            if minimum > G[m][n]:
                                minimum = G[m][n]
                                a = m
                                b = n
            #print(str(self.adj_matrix.index[a]) + "-" + str(self.adj_matrix.columns[b]) + ":" + str(G[a][b]))
            mst_edges.append(str(self.adj_matrix.index[a]) + "-" + str(self.adj_matrix.columns[b]))
            mst_cost.append(G[a][b])
            selected_node[b] = True
            no_edge += 1
        mst_df = pd.DataFrame(np.array([mst_edges, mst_cost]).T, columns=['Edge', 'Cost'])
        print('\n\nMST Edge List after applying Prim Algorithm:\n')
        display(mst_df)
        
        print(f'\nMST Cost: {mst_df.Cost.astype(int).sum()}')

## Kruskal Example

In [12]:
g = Graph(edge_list=[(1, 2, 10),(3, 6, 15), (4, 6, 20), (3, 5, 35), (1, 4, 30),
                    (2, 5, 40),(2, 6, 25),(5, 6, 55),(2, 3, 50)])

In [13]:
g.kruskal_MST()

Sorted Input Edge List:



,Edge,Cost
0,1-2,10
1,3-6,15
2,4-6,20
3,2-6,25
4,1-4,30
5,3-5,35
6,2-5,40
7,2-3,50
8,5-6,55




Applying Kruskal Algorithm


Clusters: [[1], [2], [3], [6], [4], [5]]

Edge: 1-2

1 and 2 are in different clusters

Add edge 1-2 to MST

Merge clusters of 1 and 2

Clusters: [[1, 2], [3], [6], [4], [5]]
----------------------------------------------------------

Edge: 3-6

3 and 6 are in different clusters

Add edge 3-6 to MST

Merge clusters of 3 and 6

Clusters: [[1, 2], [3, 6], [4], [5]]
----------------------------------------------------------

Edge: 4-6

4 and 6 are in different clusters

Add edge 4-6 to MST

Merge clusters of 4 and 6

Clusters: [[1, 2], [4, 3, 6], [5]]
----------------------------------------------------------

Edge: 2-6

2 and 6 are in different clusters

Add edge 2-6 to MST

Merge clusters of 2 and 6

Clusters: [[1, 2, 4, 3, 6], [5]]
----------------------------------------------------------

Edge: 1-4

1 and 4 are in same cluster, so reject edge 1-4
----------------------------------------------------------

Edge: 3-5

3 and 5 are in different clusters

Ad

,Edge,Cost
0,1-2,10
1,3-6,15
2,4-6,20
3,2-6,25
4,3-5,35



MST Cost: 105


## Prim Example - 1

In [254]:
g = Graph(edge_list=[(1, 2, 10),(3, 6, 15), (4, 6, 20), (3, 5, 35), (1, 4, 30),
                    (2, 5, 40),(2, 6, 25),(5, 6, 55),(2, 3, 50)])
g.prim_MST()

Adjacency Matrix:



,1,2,3,4,5,6
1,0,10,0,30,0,0
2,10,0,50,0,40,25
3,0,50,0,0,35,15
4,30,0,0,0,0,20
5,0,40,35,0,0,55
6,0,25,15,20,55,0




MST Edge List after applying Prim Algorithm:



,Edge,Cost
0,1-2,10
1,2-6,25
2,6-3,15
3,6-4,20
4,3-5,35



MST Cost: 105


## Prim Example - 2

In [255]:
adj_matrix = [[0, 2, 8, 0, 7,0],
   [2, 0, 5, 7, 0, 0],
   [8, 5, 0, 9, 8, 0],
   [0, 7, 9, 0, 0, 4],
   [7, 0, 8, 0, 0, 3],
           [0,0,0,4,3,0]]
g = Graph(adj_matrix= adj_matrix)

In [256]:
g.prim_MST()

Adjacency Matrix:



,0,1,2,3,4,5
0,0,2,8,0,7,0
1,2,0,5,7,0,0
2,8,5,0,9,8,0
3,0,7,9,0,0,4
4,7,0,8,0,0,3
5,0,0,0,4,3,0




MST Edge List after applying Prim Algorithm:



,Edge,Cost
0,0-1,2
1,1-2,5
2,0-4,7
3,4-5,3
4,5-3,4



MST Cost: 21


In [14]:
g = Graph(edge_list=[("A", "B", 7),("A", "C", 8),("B", "C", 3),("B", "D", 6),("C", "F", 3),("B", "F", 5),
                     ("C", "D", 4),("D", "F", 2),("D", "E", 5),("F", "E", 2)])
g.kruskal_MST()

Sorted Input Edge List:



,Edge,Cost
0,D-F,2
1,F-E,2
2,B-C,3
3,C-F,3
4,C-D,4
5,B-F,5
6,D-E,5
7,B-D,6
8,A-B,7
9,A-C,8




Applying Kruskal Algorithm


Clusters: [['A'], ['B'], ['C'], ['D'], ['F'], ['E']]

Edge: D-F

D and F are in different clusters

Add edge D-F to MST

Merge clusters of D and F

Clusters: [['A'], ['B'], ['C'], ['D', 'F'], ['E']]
----------------------------------------------------------

Edge: F-E

F and E are in different clusters

Add edge F-E to MST

Merge clusters of F and E

Clusters: [['A'], ['B'], ['C'], ['D', 'F', 'E']]
----------------------------------------------------------

Edge: B-C

B and C are in different clusters

Add edge B-C to MST

Merge clusters of B and C

Clusters: [['A'], ['B', 'C'], ['D', 'F', 'E']]
----------------------------------------------------------

Edge: C-F

C and F are in different clusters

Add edge C-F to MST

Merge clusters of C and F

Clusters: [['A'], ['B', 'C', 'D', 'F', 'E']]
----------------------------------------------------------

Edge: C-D

C and D are in same cluster, so reject edge C-D
------------------------------------------------

,Edge,Cost
0,D-F,2
1,F-E,2
2,B-C,3
3,C-F,3
4,A-B,7



MST Cost: 17
